<a href="https://colab.research.google.com/github/Mahalaxmi-png/IIITH-Wiki/blob/main/WeatherApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Create the main project folder
project_folder = '/content/weather_app'
os.makedirs(project_folder, exist_ok=True)

# Create subfolders inside the main folder
templates_folder = os.path.join(project_folder, 'templates')
static_folder = os.path.join(project_folder, 'static')

# Create subfolder for styles inside 'static'
css_folder = os.path.join(static_folder, 'css')
js_folder = os.path.join(static_folder, 'js')

# Creating the folders
os.makedirs(templates_folder, exist_ok=True)
os.makedirs(static_folder, exist_ok=True)
os.makedirs(css_folder, exist_ok=True)
os.makedirs(js_folder, exist_ok=True)

# Checking if folders are created
os.listdir(project_folder)


['templates', 'static']

In [ ]:

!pip install flask-ngrok



In [ ]:
from flask import Flask
from threading import Thread

app = Flask(__name__)

@app.route('/')
def home():
    return "Flask app is running!"

def run():
    app.run()

# Start Flask app in a thread
t = Thread(target=run)
t.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
!ngrok authtoken 2vhsyUe2nhYU5rpp1tguznTbgxC_3N7B1cpQcj9gBrNVBw1eB


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Open a tunnel on port 5000 (Flask default port)
public_url = ngrok.connect(5000)
print(f"Flask app is accessible at: {public_url}")


Flask app is accessible at: NgrokTunnel: "https://be08-34-85-252-107.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!pip install flask flask-ngrok requests
!pip install flask pyngrok --quiet


In [ ]:
!fuser -k 5000/tcp


In [ ]:
from flask import Flask, render_template_string, request
from pyngrok import ngrok
import requests

app = Flask(__name__)

# HTML for the Opening Page with Bootstrap
opening_page_html = '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weather App - Welcome</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .welcome-container {
            padding: 50px;
            text-align: center;
        }
        .btn-custom {
            background-color: #4CAF50;
            color: white;
            border-radius: 5px;
            font-size: 20px;
            padding: 12px 30px;
            text-decoration: none;
        }
        .btn-custom:hover {
            background-color: #45a049;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container welcome-container">
        <h1 class="display-4 text-success">Welcome to the Weather App</h1>
        <p class="lead">Get real-time weather updates for any city across the world!</p>
        <a href="/weather" class="btn btn-custom">Get Started</a>
    </div>
</body>
</html>
'''

# HTML for Weather Page with Bootstrap
weather_page_html = '''
<!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weather App</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha1/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .weather-container {
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            background-color: white;
            max-width: 600px;
            margin: auto;
        }
        .btn-custom {
            background-color: #4CAF50;
            color: white;
            border-radius: 5px;
            font-size: 18px;
        }
        .btn-custom:hover {
            background-color: #45a049;
        }
        .result p {
            font-size: 18px;
        }
        .error {
            color: red;
            font-weight: bold;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container weather-container">
        <h2 class="text-center text-success">Enter a City</h2>
        <form method="post" class="d-flex flex-column align-items-center">
            <div class="form-group mb-3">
                <input name="city" type="text" class="form-control" placeholder="City name" required>
            </div>
            <button type="submit" class="btn btn-custom mb-3">Get Weather</button>
        </form>
        {% if weather %}
            <div class="result text-center">
                {% if weather['name'] != 'Not Found' %}
                    <p><strong>City:</strong> {{ weather['name'] }}</p>
                    <p><strong>Temperature:</strong> {{ weather['main']['temp'] }}°C</p>
                    <p><strong>Weather:</strong> {{ weather['weather'][0]['description'] }}</p>
                {% else %}
                    <p class="error">{{ weather['weather'][0]['description'] }}</p>
                {% endif %}
            </div>
        {% endif %}
    </div>
</body>
</html>
'''

@app.route("/", methods=["GET"])
def home():
    return render_template_string(opening_page_html)

@app.route("/weather", methods=["GET", "POST"])
def weather():
    weather = None
    if request.method == "POST":
        city = request.form["city"]
        api_key = "6f4c378993088de2aa2e21fd92f60eff"  # Replace this with your OpenWeatherMap API key
        url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
        response = requests.get(url)
        if response.status_code == 200:
            weather = response.json()
        else:
            weather = {"name": "Not Found", "main": {"temp": "N/A"}, "weather": [{"description": "Invalid City"}]}
    return render_template_string(weather_page_html, weather=weather)

# 🔁 Start Flask server with ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(" * ngrok tunnel URL:", public_url)
app.run(port=port)


 * ngrok tunnel URL: NgrokTunnel: "https://94ef-34-85-252-107.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Apr/2025 05:51:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2025 05:51:44] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2025 05:51:45] "GET /weather HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2025 05:51:48] "POST /weather HTTP/1.1" 200 -
